# Speed Metrics

Loads and saves PageSpeed and page loading time.

Currently uses a local PHP script sending requests to the GT Metrix API.

A complete test may last up to 30 minutes.

For mobile device performances tested with “mobile_metrics.ipynb”. GT Metrix doesn't offer testing on mobile devices for now.

Feel free to contact me for help: https://www.quel-media.com/about.html#contact

© Paul Ronga under Apache-2 Licence (see LICENCE.txt).

In [1]:
import pandas as pd
import requests
from IPython.display import HTML
import json
import datetime

In [2]:
# change this for your local tester / an external tool
TESTER_URL = 'http://rospo.local/~paul/gtmetrix/medias.php'

In [3]:
# dataframe containing media id, name and URLs
medias = pd.read_csv('df/media_list.csv')

medias.head(2)

,media_id,Name,URL_short,URL,URL_mobile
0,19,La Tribune de Genève,tdg.ch,https://www.tdg.ch/,https://m.tdg.ch
1,20,24 heures,24heures.ch,https://www.24heures.ch,https://m.24heures.ch


In [4]:
# remove Konbini
medias = medias[medias['media_id'] < 34].copy()

# media id as string
medias['media_id'] = medias['media_id'].apply(lambda x: str(x))

In [5]:
# this new dataframe will contain our stats
df_speed = pd.DataFrame(columns=['Name', 'media_id', 'pagespeed_score', 'page_load_time', 'fully_loaded_time', 'report_url'])

In [6]:
for i, row in medias[1:].iterrows():
    print('Testing', row['Name'], '...')
    media_index = i

    payload = {'media': medias.loc[media_index][['Name', 'media_id', 'URL']].to_dict()}
    r = requests.post(TESTER_URL, json=payload)
    
    print(r.text, end='\n\n')
    
    result = json.loads(r.text.split('\n')[-1])

    df_speed = df_speed.append(pd.DataFrame([[
        result['media']['Name'],
        result['media']['media_id'],
        result['results']['pagespeed_score'],
        result['results']['page_load_time'] / 1000,
        result['results']['fully_loaded_time'] / 1000,
        result['results']['report_url']
    ]], columns=['Name', 'media_id', 'pagespeed_score', 'page_load_time', 'fully_loaded_time', 'report_url']))

Testing 24 heures ...
Test started with nmQgdyxV
{"media":{"media_id":"20","Name":"24 heures","URL":"https:\/\/www.24heures.ch"},"results":{"onload_time":10413,"first_contentful_paint_time":5121,"page_elements":310,"report_url":"https:\/\/gtmetrix.com\/reports\/www.24heures.ch\/Ut8ewfQX","redirect_duration":0,"first_paint_time":5121,"dom_content_loaded_duration":null,"dom_content_loaded_time":7205,"dom_interactive_time":7205,"page_bytes":8114104,"page_load_time":10413,"html_bytes":42160,"fully_loaded_time":10732,"html_load_time":967,"rum_speed_index":5126,"yslow_score":41,"pagespeed_score":33,"backend_duration":737,"onload_duration":65,"connect_duration":230}}

Testing Le Temps ...
Test started with E5xnLkdD
{"media":{"media_id":"21","Name":"Le Temps","URL":"https:\/\/www.letemps.ch"},"results":{"onload_time":13029,"first_contentful_paint_time":5818,"page_elements":262,"report_url":"https:\/\/gtmetrix.com\/reports\/www.letemps.ch\/CbvVCgYY","redirect_duration":0,"first_paint_time":5818

Test started with u4ZYO1k9
{"media":{"media_id":"33","Name":"Le Courrier","URL":"https:\/\/lecourrier.ch\/"},"results":{"onload_time":8959,"first_contentful_paint_time":7551,"page_elements":61,"report_url":"https:\/\/gtmetrix.com\/reports\/lecourrier.ch\/ms1q19xQ","redirect_duration":0,"first_paint_time":7291,"dom_content_loaded_duration":null,"dom_content_loaded_time":8389,"dom_interactive_time":8389,"page_bytes":1705554,"page_load_time":8959,"html_bytes":24557,"fully_loaded_time":9668,"html_load_time":6121,"rum_speed_index":7543,"yslow_score":80,"pagespeed_score":89,"backend_duration":5631,"onload_duration":13,"connect_duration":490}}



In [7]:
# Use this in case you get e.g. a “The page took too long to load” or “Unable to analyze your site” error.
# It will contain missing medias. You can loop through it in the previous cell.
missing_medias = medias[(-medias['media_id'].isin(df_speed['media_id']))]

In [8]:
missing_medias.head(3)

,media_id,Name,URL_short,URL,URL_mobile
0,19,La Tribune de Genève,tdg.ch,https://www.tdg.ch/,https://m.tdg.ch


In [9]:
# To check for a report after an error
print("https:\/\/gtmetrix.com\/reports\/www.lacote.ch\/ZrEyp4s4".replace('\\', ''))

https://gtmetrix.com/reports/www.lacote.ch/ZrEyp4s4


In [10]:
# add current timestamp
df_speed['timestamp'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_speed

,Name,media_id,pagespeed_score,page_load_time,fully_loaded_time,report_url,timestamp
0,24 heures,20,33,10.413,10.732,https://gtmetrix.com/reports/www.24heures.ch/U...,2018-07-19 17:26:26
0,Le Temps,21,49,13.029,14.714,https://gtmetrix.com/reports/www.letemps.ch/Cb...,2018-07-19 17:26:26
0,Le Monde,22,41,8.333,8.734,https://gtmetrix.com/reports/www.lemonde.fr/sw...,2018-07-19 17:26:26
0,RTS info,23,59,6.365,7.074,https://gtmetrix.com/reports/www.rts.ch/3vATvP8M,2018-07-19 17:26:26
0,20 minutes (ch),24,20,10.587,11.243,https://gtmetrix.com/reports/www.20min.ch/Qdjy...,2018-07-19 17:26:26
0,Le Matin,25,0,12.322,13.414,https://gtmetrix.com/reports/www.lematin.ch/KR...,2018-07-19 17:26:26
0,Mediapart,26,25,7.304,7.901,https://gtmetrix.com/reports/www.mediapart.fr/...,2018-07-19 17:26:26
0,Le Figaro,27,32,10.100,30.565,https://gtmetrix.com/reports/www.lefigaro.fr/C...,2018-07-19 17:26:26
0,Libération,28,3,31.920,35.212,https://gtmetrix.com/reports/www.liberation.fr...,2018-07-19 17:26:26
0,La Côte,29,41,11.032,17.234,https://gtmetrix.com/reports/www.lacote.ch/gPV...,2018-07-19 17:26:26


In [11]:
outputfile = 'df/archive/speed_metrics_{}.csv'.format( datetime.datetime.now().strftime('%Y-%m-%d') )
print('Saving to {}...'.format(outputfile))

Saving to df/archive/speed_metrics_2018-07-19.csv...


In [12]:
df_speed.to_csv(outputfile) # archive
df_speed.to_csv('df/speed_metrics.csv') # temp file